In [0]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline
import missingno as msno
from datetime import timedelta


KeyboardInterrupt: 

In [ ]:
os.getcwd()
os.chdir('C:\\Users\\IMEN\\Documents\\openclassroom\\formation openclassrooms\\projet5')

### 1.Présentation des données

In [ ]:
csv_files = glob.glob(os.path.join('datasets', "*.csv"))
csv_files

In [ ]:
#9 dataframe (df1..df9)
for i in range(len(csv_files)):   
    data=pd.read_csv(csv_files[i])
    globals()[f'df{i+1}'] =data
    

In [ ]:
from IPython.display import Image
Image(filename='HRhd2Y0.png',width = 600, height = 600)

In [ ]:
datasets=['olist_customers_dataset','olist_geolocation_dataset','olist_orders_dataset',
 'olist_order_items_dataset.csv','olist_order_payments_dataset','olist_order_reviews_dataset',
 'olist_products_dataset','olist_sellers_dataset']
Descriptions=['informations sur le client et son emplacement',
             ' informations sur les codes postaux brésiliens et leurs coordonnées','indormations sur les commandes',
             'données sur les articles achetés dans chaque commande','données sur les options de paiement des commandes',
             'données sur les évaluations faites par les clients',
             'données sur les produits vendus par Olist','données sur les vendeurs qui ont exécuté les commandes passées sur Olist']
dic={'Variable':['df1','df2','df3','df4','df5','df6','df7','df8'],'Dataset': datasets, 'Description': Descriptions}
pd.set_option("display.max_colwidth", None,'display.colheader_justify', 'center')
data_description=pd.DataFrame(dic)
data_description

In [ ]:
ds=[df1,df2,df3,df4,df5,df6,df7,df8]

In [ ]:

for d in ds:
    print(d.columns)
    print('-'*20)
    print(d.describe())
    print('*'*100)

#### Dataframes et valeurs nulles

In [ ]:
#msno.bar(df1):ok
#msno.bar(df2):ok
#msno.bar(df3)#contient des nan
#msno.bar(df4)#ok
#msno.bar(df5)#ok
#msno.bar(df6)#contient des nan
#msno.bar(df7)#contient des nan
#msno.bar(df8)#ok

In [ ]:
#supprimer les lignes contenant des nan
df3.dropna(axis=0,inplace=True)
df7.dropna(axis=0,inplace=True)
#pour df6 les colonnes contenant des nan sont review_comment_title et review_content_message ==>je supprime ces deux colonnes
df6.dropna(axis=1,inplace=True)

In [ ]:

for d in ds:
    print(d.columns)
    print('\n')



#### Dataframes et lignes dupliquées

In [ ]:
#print('Entrées dupliquées: {}'.format(df1.duplicated().sum()))#0
print('Entrées dupliquées: {}'.format(df2.duplicated().sum()))
df2.drop_duplicates(inplace = True)
#print('Entrées dupliquées: {}'.format(df3.duplicated().sum()))#0
#print('Entrées dupliquées: {}'.format(df4.duplicated().sum()))#0
#print('Entrées dupliquées: {}'.format(df5.duplicated().sum()))#0
#print('Entrées dupliquées: {}'.format(df6.duplicated().sum()))#0
#print('Entrées dupliquées: {}'.format(df7.duplicated().sum()))#0
#print('Entrées dupliquées: {}'.format(df8.duplicated().sum()))#0

#### Dataframes et suppression de lignes

In [ ]:
#supprimer les commnandes annulées et non valables
df3=df3[df3.order_status!='canceled']
df3=df3[df3.order_status!='unavailable']

In [ ]:
df3.order_status.unique()

### Analyse exploratoire

#### Nombre de commande par clients

In [ ]:
#nbre de commandes par client
nb_order_per_customer=pd.DataFrame(df1.groupby('customer_unique_id')['customer_id'].count())\
.rename(columns={'customer_id':'Nb'}).reset_index()
nb_order_per_customer

In [ ]:
#Clients avec plusieurs commandes
customer_with_many_orders=nb_order_per_customer[nb_order_per_customer['Nb']>1]
per_cust_with_many_orders=round(100*customer_with_many_orders.shape[0]/nb_order_per_customer.shape[0],2)
print(f'{per_cust_with_many_orders} % des clients ayant réalisés plusieurs commandes')

In [ ]:
#Clients avec une seule commande
customer_with_one_order=nb_order_per_customer[nb_order_per_customer['Nb']==1]
per_cust_with_one_order=round(100*customer_with_one_order.shape[0]/nb_order_per_customer.shape[0],2)
print(f'{per_cust_with_one_order} % des clients ayant réalisés une seule commande')

In [ ]:
df12=df2.join(df1.set_index(['customer_zip_code_prefix','customer_city','customer_state']), on=['geolocation_zip_code_prefix', 'geolocation_city','geolocation_state'], how='right')
df12

In [ ]:
# Géolocalisation des clients
# map = folium.Map()
# map = folium.Map(location=[df12['geolocation_lat'].mean(), df12['geolocation_lng'].mean()],zoom_start=10)
# for i in range(0,len(df12)):
#     folium.Circle([df12.iloc[i]['geolocation_lat'],df12.iloc[i]['geolocation_lng']], 
#                       popup=df12.iloc[i]['geolocation_city'], radius =100).add_to(map)
# map

#### Montants des commandes

In [ ]:
df4.price.describe()#Pour vérifier s'il ya des prix négatifs

In [ ]:
#____________________
# Décompte des achats
price_range = [0, 50, 100, 200, 500, 1000, 10000]
count_price = []
for i, price in enumerate(price_range):
    if i == 0: continue
    val = df4[(df4.price < price) &
                       (df4.price > price_range[i-1])]['price'].count()
    count_price.append(val)
#____________________________________________
# Représentation du nombre d'achats / montant        
plt.rc('font', weight='bold')
f, ax = plt.subplots(figsize=(11, 6))
colors = ['yellowgreen', 'gold', 'wheat', 'c', 'violet', 'royalblue','firebrick']
labels = [ '{}<.<{}'.format(price_range[i-1], s) for i,s in enumerate(price_range) if i != 0]
sizes  = count_price
explode = [0.0 if sizes[i] < 100 else 0.0 for i in range(len(sizes))]

ax.pie(sizes, explode = explode, labels=labels, colors = colors,
       autopct = lambda x:'{:1.0f}%'.format(x) if x > 1 else '',
       shadow = False, startangle=0)
ax.axis('equal')
f.text(0.5, 1.01, "Répartition des montants des commandes", ha='center', fontsize = 18);

#### Catégories des produits

In [ ]:
df7.head()

In [ ]:
#Distribution des catégories des produits 
product_category=pd.DataFrame(df7.groupby('product_category_name')['product_category_name'].count().reset_index(name='counts'))
product_category=product_category.sort_values(by='counts',ascending=True)
product_category

In [ ]:
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
plt.style.use('ggplot')

font_title = {'family': 'serif',
              'color':  '#114b98',
              'weight': 'bold',
              'size': 18,
             }
fig = plt.figure(figsize=(21,40))
product_category.plot( y='counts', x='product_category_name',kind='barh',figsize=(20,20))
plt.title('Distribution des catégories de produits',fontdict=font_title)


#### Création du datset commandes

In [ ]:
#Ici on crée un dataset nommée commande_dataset contenant toute les information d'une commande

#jointure entre df1 et df3 (dataset products et dataset order)
df13=pd.merge(df1,df3,on='customer_id')
df13=df13[['customer_id', 'customer_unique_id','customer_city', 'customer_state', 'order_id', 'order_status',
       'order_purchase_timestamp']]
#df13.head()

In [ ]:

df134=pd.merge(df13,df4,on='order_id')
df134=df134[['customer_id', 'customer_unique_id', 'customer_city', 'customer_state',
       'order_id', 'order_status', 'order_purchase_timestamp', 'order_item_id',
       'product_id', 'seller_id','price']]
#df134.head()

In [ ]:

df1345=pd.merge(df134,df5,on='order_id')
df1345=df1345[['customer_id', 'customer_unique_id', 'customer_city', 'customer_state',
       'order_id', 'order_status', 'order_purchase_timestamp', 'order_item_id',
       'product_id', 'seller_id', 'price','payment_type']]
#df1345.head()

In [ ]:
df13456=pd.merge(df1345,df6,on='order_id')
df13456=df13456[['customer_id', 'customer_unique_id', 'customer_city', 'customer_state',
       'order_id', 'order_status', 'order_purchase_timestamp', 'order_item_id',
       'product_id', 'seller_id', 'price', 'payment_type', 'review_id',
       'review_score']]
#df13456.head()

In [ ]:
df134567=pd.merge(df13456,df7,on='product_id')
df134567=df134567[['customer_id', 'customer_unique_id', 'customer_city', 'customer_state',
       'order_id', 'order_status', 'order_purchase_timestamp', 'order_item_id',
       'product_id', 'seller_id', 'price', 'payment_type', 'review_id',
       'review_score', 'product_category_name']]
#df134567.head()

In [ ]:
df1345678=pd.merge(df134567,df8,on='seller_id')
#df1345678.columns

In [ ]:
commandes=df1345678

In [ ]:
msno.bar(commandes)

## Sauvegarde des données

In [ ]:
#print('Entrées dupliquées: {}'.format(commandes.duplicated().sum()))
commandes.drop_duplicates(inplace = True)

In [ ]:
#Ici je vais garder le data set localisation df2
df2.to_csv('geolocalisation_dataset.csv',index=False)
commandes.to_csv('commandes_dataset.csv',index=False)
#df2.to_csv('geolocalisation_dataset.csv', sep='\t',index=False)